In [3]:
import os
import requests
import docker
import json
import opendatasets as od

E:\Uni\pwr-pdzb\vagrant\sprawozdania\akwizycja


In [4]:
client = docker.from_env()
container = client.containers.get('master')

def hdfs_mkdir(path):
    container.exec_run(f"hdfs dfs -mkdir -p /{path}/")

def hdfs_upload(path):
    directory = "/".join(path.split("/")[:-1])
    hdfs_mkdir(directory)
    cmd = f"hdfs dfs -put /data/master_volume/{path} /{directory}"
    print(cmd)
    code, output = container.exec_run(cmd)
    print(f"exit code {code}")
    print(output)

def hdfs_set_replication_level(number):
    container.exec_run(f"hdfs dfs -setrep -R {number} /")

Ensure `kaggle.json` is located in the root of the repository or at `~/.kaggle/kaggle.json`.

In [5]:
output_dir = "../../master_volume/datasets"

In [6]:
if not os.path.isdir(f"{output_dir}/steam-dataset"):
    od.download("https://www.kaggle.com/datasets/souyama/steam-dataset", f"{output_dir}")
else:
    print("Dataset steam-dataset already exists, skipping download")

if not os.path.isdir(f"{output_dir}/youtube-trending-video-dataset"):
    od.download("https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset", f"{output_dir}")
else:
    print("Dataset youtube-trending-video-dataset already exists, skipping download")

Dataset steam-dataset already exists, skipping download
Dataset youtube-trending-video-dataset already exists, skipping download


In [7]:
path = f"{output_dir}/covid-dataset.csv"

if not os.path.isfile(path):
    print(f"Downloading covid-dataset to {path}")
    r = requests.get("https://covid.ourworldindata.org/data/owid-covid-data.csv", allow_redirects=True)
    with open(path, 'wb') as file:
        file.write(r.content)
else:
    print("Dataset covid-dataset already exists, skipping download")

print(f"covid-dataset.csv: {os.stat(path).st_size / (1024 * 1024):.02f}MB")

Dataset covid-dataset already exists, skipping download
covid-dataset.csv: 77.76MB


In [9]:
print("Converting JSON to JSONL")
for root, directories, files in os.walk(f"{output_dir}"):
        for filename in files:
            path = os.path.join(root,filename)
            output_path = f"{path}l"

            if os.path.isfile(output_path):
                continue

            if path.endswith(".json"):
                print(path)
                with open(path, "r") as file:
                    data = json.load(file)
                    if type(data) is dict:
                        data = [{"key": key, "value": data[key]} for key in data]

                    with open(output_path, "w") as jsonl_file:
                        for obj in data:
                            json.dump(obj, jsonl_file)
                            jsonl_file.write("\n")
                    print(output_path)

print("Done")

Converting JSON to JSONL
Done


In [11]:
print("Uploading to HDFS")
for root, directories, files in os.walk(f"{output_dir}"):
        for filename in files:
            if filename.endswith(".json"):
                continue # skip JSON files, we have JSONL from previous step
            path = os.path\
                .join(root,filename)\
                .removeprefix("../../master_volume/")\
                .replace("\\", "/")

            hdfs_upload(path)
print("Done")

Uploading to HDFS
datasets/covid-dataset.csv
datasets/steam-dataset/steam_dataset/appinfo/dlc_data/missing.jsonl
datasets/steam-dataset/steam_dataset/appinfo/dlc_data/steam_dlc_data.jsonl
datasets/steam-dataset/steam_dataset/appinfo/dlc_data/timestamp.txt
datasets/steam-dataset/steam_dataset/appinfo/store_data/steam_store_data.jsonl
datasets/steam-dataset/steam_dataset/appinfo/store_data/timestamp.txt
datasets/steam-dataset/steam_dataset/news_data/missing.jsonl
datasets/steam-dataset/steam_dataset/news_data/steam_news_data.jsonl
datasets/steam-dataset/steam_dataset/news_data/timestamp.txt
datasets/steam-dataset/steam_dataset/steamspy/basic/steam_spy_scrap.jsonl
datasets/steam-dataset/steam_dataset/steamspy/basic/timestamp.txt
datasets/steam-dataset/steam_dataset/steamspy/detailed/steam_spy_detailed.jsonl
datasets/steam-dataset/steam_dataset/steamspy/detailed/timestamp.txt
datasets/steam-dataset/steam_dataset/steam_charts/missing.jsonl
datasets/steam-dataset/steam_dataset/steam_charts/s

In [56]:
hdfs_set_replication_level(3)